<a href="https://colab.research.google.com/github/matherdy/Ukraine-War-Reserach/blob/krutik-2-11-collected-tweets/Tweets_Fetch_Using_Essential_Access_and_storing_in_AWS_S3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Python code to fetch tweets from Twitter using essential access and storing it to AWS S3. The script also includes the code to fetch data from S3 and using it in a python dataframe.


In [1]:
!pip install --upgrade tweepy 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 3.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0


In [2]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 KB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 37.4 MB/s eta 0:00:00


In [51]:
import tweepy
import csv
import boto3
import pandas as pd
from io import StringIO


TWEET_FIELDS = [
    'id',
    'text',
    'created_at'
]

# AWS credentials
access_key = "**********"
secret_key = "**********"


# Twitter API credentials
BEARER_TOKEN: str = "**********"

query = "russia+ukraine"

CLIENT = tweepy.Client(bearer_token=BEARER_TOKEN)

response = CLIENT.search_recent_tweets(query=query,tweet_fields=TWEET_FIELDS,max_results=100)

tweets = []

tweets += [tweet.data for tweet in response[0]]

filename = f'{query}_tweets.csv' 

# Create a CSV file to store the tweets
with open(filename, 'w') as file:
    writer = csv.writer(file, lineterminator = "\n")
    writer.writerow(['Keyword', 'tweet_id', 'created_at','text'])

    # Iterate over the tweets and write them to the CSV file
    for tweet in tweets:
        writer.writerow([query,tweet['id'],tweet['created_at'],tweet['text']])

print("***************** Tweets added in CSV file **************************")



filename = 'russia+ukraine_tweets.csv'

# Authenticate to AWS
s3 = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key)

bucket_name = 'russiaukrainewarproject'
key = filename #path to file

################### Upload the CSV file to S3##########################
s3.upload_file(Filename=filename, Bucket=bucket_name, Key=key)
print("**************** CSV file uploaded to AWS S3 *******************")


print("**************** Reading CSV file from AWS S3 ******************")
#read csv file from S3
response = s3.get_object(Bucket=bucket_name, Key=key)

status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

if status == 200:
    print(f"************** Successful S3 get_object response. Status - {status } ***********************")
    body = response['Body']
    csv_string = body.read().decode()
    df = pd.read_csv(StringIO(csv_string))
    print(df)
else:
    print(f"************ Unsuccessful S3 get_object response. Status - {status} ***********************")


**************** CSV file uploaded to AWS S3 *******************
**************** Reading CSV file from AWS S3 ******************
************** Successful S3 get_object response. Status - 200 ***********************
           Keyword             tweet_id                created_at  \
0   russia+ukraine  1631797504894197762  2023-03-03T23:23:37.000Z   
1   russia+ukraine  1631797452935159814  2023-03-03T23:23:24.000Z   
2   russia+ukraine  1631797359536488448  2023-03-03T23:23:02.000Z   
3   russia+ukraine  1631797325566820357  2023-03-03T23:22:54.000Z   
4   russia+ukraine  1631797310983397376  2023-03-03T23:22:50.000Z   
..             ...                  ...                       ...   
95  russia+ukraine  1631794386676731907  2023-03-03T23:11:13.000Z   
96  russia+ukraine  1631794333387980800  2023-03-03T23:11:01.000Z   
97  russia+ukraine  1631794317063774208  2023-03-03T23:10:57.000Z   
98  russia+ukraine  1631794312634863616  2023-03-03T23:10:56.000Z   
99  russia+ukraine  1631